In [56]:
%cd /Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/

/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt


In [57]:
import tensorflow as tf

In [58]:
# GLobal definations

path = '/Users/vitthal/Documents/Research/DataScience/MedicalResearch/mgmt/'
block_h, block_w = (32, 32)
Sub_Model = 5

In [59]:
# Function Definition --> Loading preprocessed data from the file & Normalize it.

def Load_Data():
    pickle_in = open(Case_Path + '/' + 'X_train.pickle', 'rb')
    X = pickle.load(pickle_in)  # Loading Features

    pickle_in = open(Case_Path + '/' + 'y_train.pickle', 'rb')
    y = pickle.load(pickle_in)  # Loading Labels

    print('Shape of X:\t', X.shape), print('Shape of y:\t', y.shape)

    return X, y

In [60]:
# Function Definition --> For Development, Compilation, Training & Storing of the Model

def Model(img_h, img_w, X_train, y_train, X_test, y_test, count):
    # import tensorflow as tf
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, LeakyReLU, Activation
    from tensorflow.keras.utils import to_categorical

    model = Sequential()

    model.add (Conv2D (32, (3, 3), padding = 'same', input_shape = (img_h, img_w, 1)))
    model.add (LeakyReLU (alpha = 0.1))
    model.add (MaxPooling2D (pool_size = (3, 3)))  # Pool_size is the size of filter.
    model.add (Dropout(0.1))

    model.add (Conv2D (48, (3, 3), padding = 'same'))
    model.add (LeakyReLU (alpha = 0.1))
    model.add (MaxPooling2D (pool_size = (3, 3)))
    model.add (Dropout(0.1))

    model.add (Conv2D (48, (3, 3), padding = 'same'))
    model.add (LeakyReLU (alpha = 0.1))
    model.add (MaxPooling2D (pool_size = (3, 3)))
    model.add (Dropout(0.1))

    model.add (Flatten())  # Convert 3D feature map to 1D feature vector.

    model.add (Dense(1096))
    model.add (LeakyReLU (alpha = 0.1))
    model.add (Dropout(0.1))
    model.add (Dense(2, activation = 'softmax'))

    # Compiling the model
    model.compile (loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    model.optimizer.learning_rate = 0.0001

    # Training the model & storing its detail in the variable h
    history = model.fit (X_train, to_categorical(y_train), batch_size = 128, epochs = 50, verbose = 2,
               validation_data=(X_test, to_categorical(y_test)), shuffle = True)

    # Evaluating performance of the Model
    test_loss, test_acc = model.evaluate (X_test, to_categorical(y_test))
    results.append(test_acc)
    print('Test Loss is: ', test_loss, '\nTest Accuracy is: ', test_acc)

    # Saving the model in hierarchical Data Formate (HDF)
    model.save (path + 'Outputs/Model_' + case + '_' + str(count+1) + '.h5')

    return history, model

In [61]:
type(X_train)

numpy.ndarray

In [62]:
# Function Definition --> For Saving Validation Accuracy in the file

def Saving_Test_Accuracy():
    import numpy as np
    
    file = open(Result_File, "a")
    file.write(case)
    file.write(':\n======')
    file.write('\nAll Test Accuracy:\t')
    
    for ans in results:
        file.write(str(ans) + ', ')
    
    file.write('\nMean Test Accuracy:\t' + str(np.mean(results)))
    file.write('\nMin Test Accuracy:\t' + str(np.min(results)))
    file.write('\nMax Test Accuracy:\t' + str(np.max(results)))
    file.write('\n\n\n')
    file.close()

In [63]:
# All Function Calls
import os
from tqdm import tqdm
import pickle
from sklearn.model_selection import train_test_split
from random import randint
import numpy as np

PATH = path + 'Data/UPENN/'
Work_Dir = PATH + 'Working_Data/'
Result_File = path + 'Outputs/Results.txt'

# Calling Functions:
osdir=os.listdir(Work_Dir)
if '.DS_Store' in osdir:
    osdir.remove('.DS_Store')

for case in tqdm(osdir):
    Case_Path = os.path.join(Work_Dir, case)  # Joining path for "Cases'.
    print(case)

    # Function Call --> Loading preprocessed data from the file & Normalize it.
    X, y = Load_Data()
    print(type(X))
    print(type(y))
    # X= X.astype(np.float32)
    # y=y.astype(np.float32)
    # print(type(X))
    # print(type(y))

    results = []
    for i in range(Sub_Model):
        print('\n' + case + '_' + str(i+1))
        
        # Spliting data into two parts, "Training = 70%" and "Testing = 30%".
        num = randint(0,1000)
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = num)

        # Function Call --> For Development, Compilation, Training & Storing of the Model
        history, model = Model(block_h, block_w, X_train, y_train, X_test, y_test, i)
    print('\n')

    # Function Call --> For Saving Validation Accuracy in the file
    Saving_Test_Accuracy()
print('Done...!!')

  0%|          | 0/5 [00:00<?, ?it/s]

Case_5
Shape of X:	 (15815, 32, 32, 1)
Shape of y:	 (15815,)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>

Case_5_1
Epoch 1/50


2022-09-07 23:43:36.282652: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-07 23:43:39.595672: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


87/87 - 4s - loss: 0.9828 - accuracy: 0.5835 - val_loss: 0.6496 - val_accuracy: 0.6434 - 4s/epoch - 49ms/step
Epoch 2/50
87/87 - 2s - loss: 0.5885 - accuracy: 0.6965 - val_loss: 0.4980 - val_accuracy: 0.7418 - 2s/epoch - 20ms/step
Epoch 3/50
87/87 - 2s - loss: 0.5271 - accuracy: 0.7373 - val_loss: 0.4533 - val_accuracy: 0.7730 - 2s/epoch - 20ms/step
Epoch 4/50
87/87 - 2s - loss: 0.4742 - accuracy: 0.7696 - val_loss: 0.3750 - val_accuracy: 0.8434 - 2s/epoch - 20ms/step
Epoch 5/50
87/87 - 2s - loss: 0.4303 - accuracy: 0.8005 - val_loss: 0.3436 - val_accuracy: 0.8443 - 2s/epoch - 20ms/step
Epoch 6/50
87/87 - 2s - loss: 0.3918 - accuracy: 0.8201 - val_loss: 0.2930 - val_accuracy: 0.8769 - 2s/epoch - 21ms/step
Epoch 7/50
87/87 - 2s - loss: 0.3484 - accuracy: 0.8449 - val_loss: 0.2713 - val_accuracy: 0.8944 - 2s/epoch - 20ms/step
Epoch 8/50
87/87 - 2s - loss: 0.3135 - accuracy: 0.8612 - val_loss: 0.2369 - val_accuracy: 0.9018 - 2s/epoch - 20ms/step
Epoch 9/50
87/87 - 2s - loss: 0.2959 - accu

2022-09-07 23:45:11.602341: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-07 23:45:15.117950: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

87/87 - 3s - loss: 0.9885 - accuracy: 0.5799 - val_loss: 0.5614 - val_accuracy: 0.6969 - 3s/epoch - 38ms/step
Epoch 2/50
87/87 - 2s - loss: 0.5960 - accuracy: 0.6865 - val_loss: 0.5641 - val_accuracy: 0.6917 - 2s/epoch - 24ms/step
Epoch 3/50
87/87 - 2s - loss: 0.5119 - accuracy: 0.7461 - val_loss: 0.4466 - val_accuracy: 0.7886 - 2s/epoch - 22ms/step
Epoch 4/50
87/87 - 2s - loss: 0.4771 - accuracy: 0.7690 - val_loss: 0.4020 - val_accuracy: 0.8344 - 2s/epoch - 22ms/step
Epoch 5/50
87/87 - 2s - loss: 0.4285 - accuracy: 0.7988 - val_loss: 0.3469 - val_accuracy: 0.8575 - 2s/epoch - 22ms/step
Epoch 6/50
87/87 - 2s - loss: 0.3890 - accuracy: 0.8195 - val_loss: 0.3225 - val_accuracy: 0.8691 - 2s/epoch - 21ms/step
Epoch 7/50
87/87 - 2s - loss: 0.3515 - accuracy: 0.8427 - val_loss: 0.2709 - val_accuracy: 0.8942 - 2s/epoch - 22ms/step
Epoch 8/50
87/87 - 2s - loss: 0.3165 - accuracy: 0.8611 - val_loss: 0.2730 - val_accuracy: 0.8809 - 2s/epoch - 22ms/step
Epoch 9/50
87/87 - 2s - loss: 0.2955 - accu

2022-09-08 00:20:16.297907: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-08 00:20:18.799377: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


87/87 - 3s - loss: 1.0600 - accuracy: 0.5720 - val_loss: 0.5653 - val_accuracy: 0.7026 - 3s/epoch - 38ms/step
Epoch 2/50
87/87 - 2s - loss: 0.6021 - accuracy: 0.6835 - val_loss: 0.5295 - val_accuracy: 0.7109 - 2s/epoch - 22ms/step
Epoch 3/50
87/87 - 2s - loss: 0.5273 - accuracy: 0.7371 - val_loss: 0.4443 - val_accuracy: 0.7850 - 2s/epoch - 23ms/step
Epoch 4/50
87/87 - 2s - loss: 0.4806 - accuracy: 0.7692 - val_loss: 0.4057 - val_accuracy: 0.8042 - 2s/epoch - 22ms/step
Epoch 5/50
87/87 - 2s - loss: 0.4329 - accuracy: 0.7956 - val_loss: 0.3389 - val_accuracy: 0.8569 - 2s/epoch - 21ms/step
Epoch 6/50
87/87 - 2s - loss: 0.3975 - accuracy: 0.8190 - val_loss: 0.3121 - val_accuracy: 0.8740 - 2s/epoch - 22ms/step
Epoch 7/50
87/87 - 2s - loss: 0.3656 - accuracy: 0.8340 - val_loss: 0.2731 - val_accuracy: 0.8938 - 2s/epoch - 23ms/step
Epoch 8/50
87/87 - 2s - loss: 0.3230 - accuracy: 0.8584 - val_loss: 0.2842 - val_accuracy: 0.8801 - 2s/epoch - 22ms/step
Epoch 9/50
87/87 - 2s - loss: 0.3112 - accu

100%|██████████| 5/5 [38:18<00:00, 459.79s/it]



Done...!!
